# Исследование надежности заемщиков


Заказчик - кредитный отдел банка. Необходимо разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка - статистика платежеспособности клиентов.

## Обзор данных

**Импорт библиотеки pandas. Считывание данных из csv-файла в датафрейм и сохранение в переменную `data`.**

In [1]:
import pandas as pd

try:
    data = pd.read_csv('/datasets/data.csv')
except:
    data = pd.read_csv('https://code.s3.yandex.net/datasets/data.csv')

**Вывод первых 40 строчек датафрейма `data` на экран.**

In [2]:
data.head(40)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**Вывод основной информации о датафрейме с помощью метода `info()`.**

In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## Предобработка данных

### Удаление пропусков

**Вывод количества пропущенных значений для каждого столбца.**

In [4]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

**Заполнение пропусков в столбце `total_income` медианным значением по каждому типу из столбца `income_type`.**

In [5]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

### Обработка аномальных значений

**Замена всех отрицательных значений положительными в столбце `days_employed` с помощью метода `abs()`.**

In [6]:
data['days_employed'] = data['days_employed'].abs()

**Вывод медианное значение трудового стажа `days_employed` в днях для каждого типа занятости.**

In [7]:
data.groupby('income_type')['days_employed'].agg('median')

income_type
безработный        366413.652744
в декрете            3296.759962
госслужащий          2689.368353
компаньон            1547.382223
пенсионер          365213.306266
предприниматель       520.848083
сотрудник            1574.202821
студент               578.751554
Name: days_employed, dtype: float64

У двух типов (безработные и пенсионеры) получатся аномально большие значения. Исправить такие значения сложно, поэтому оставьте их как есть. Тем более этот столбец не понадобится вам для исследования.

**Вывод перечня уникальных значений столбца `children`.**

In [8]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

**Удаление аномалий в столбце `children`.**

In [9]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

**Вывод перечня уникальных значений столбца `children`, чтобы убедиться, что артефакты удалены.**

In [10]:
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

### Удаление пропусков (продолжение)

**Заполнение пропусков в столбце `days_employed` медианными значениями по каждого типа занятости `income_type`.**

In [11]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

**Вывод количества пропущенных значений для каждого столбца.**

In [12]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### Изменение типов данных

**Замена вещественного типа данных в столбце `total_income` на целочисленный.**

In [13]:
data['total_income'] = data['total_income'].astype(int)

### Обработка дубликатов

**Обработка неявных дубликатов в столбце `education`. Приведение их к нижнему регистру.**

In [14]:
data['education'] = data['education'].str.lower()

**Вывод на экран количества строк-дубликатов в данных.**

In [15]:
data.duplicated().sum()

71

In [16]:
data = data.drop_duplicates()

### Категоризация данных

**Создание в датафрейме `data` столбца `total_income_category` с категориями:**

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 и выше — `'A'`.

In [17]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [18]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

**Вывод на экран перечня уникальных целей взятия кредита из столбца `purpose`.**

In [19]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

**Создание функцию, которая на основании данных из столбца `purpose` сформирует новый столбец `purpose_category`, в который войдут следующие категории:**

- `'операции с автомобилем'`,
- `'операции с недвижимостью'`,
- `'проведение свадьбы'`,
- `'получение образования'`.

In [20]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [21]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

### Исследование данных

#### Есть ли зависимость между количеством детей и возвратом кредита в срок?

In [22]:
data_pivot_children = data.pivot_table(index=['children'], values='debt', aggfunc=['count','sum', 'mean']) # сводная таблица по кол-ву детей
data_pivot_children

,count,sum,mean
,debt,debt,debt
children,,,
0,14091,1063,0.075438
1,4808,444,0.092346
2,2052,194,0.094542
3,330,27,0.081818
4,41,4,0.097561
5,9,0,0.000000


**Вывод:** Можно сказать, что между количеством детей и возвратом кредита в срок нет прямой зависимости 

#### Есть ли зависимость между семейным положением и возвратом кредита в срок?

In [23]:
data_pivot_family = data.pivot_table(index=['family_status'], values='debt', aggfunc=['count','sum', 'mean']) # сводная таблица по семейному положению
data_pivot_family

,count,sum,mean
,debt,debt,debt
family_status,,,
Не женат / не замужем,2796,273,0.097639
в разводе,1189,84,0.070648
вдовец / вдова,951,63,0.066246
гражданский брак,4134,385,0.093130
женат / замужем,12261,927,0.075606


**Вывод:** Люди, не находящиеся в законных отношениях, возвращают кредит в срок хуже других категорий

#### Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

In [24]:
data_pivot_total_income = data.pivot_table(index=['total_income_category'], values='debt', aggfunc=['count','sum', 'mean']) # сводная таблица по уровню дохода
data_pivot_total_income

,count,sum,mean
,debt,debt,debt
total_income_category,,,
A,25,2,0.080000
B,5014,354,0.070602
C,15921,1353,0.084982
D,349,21,0.060172
E,22,2,0.090909


**Вывод:** Люди, относящиеся к категории дохода В, с большей вероятностью вернут кредит в срок, чем люди с категорией дохода С 

#### Как разные цели кредита влияют на его возврат в срок?

In [25]:
data_pivot_purpose_category = data.pivot_table(index=['purpose_category'], values='debt', aggfunc=['count','sum', 'mean']) # сводная таблица по целям кредита
data_pivot_purpose_category

,count,sum,mean
,debt,debt,debt
purpose_category,,,
операции с автомобилем,4279,400,0.093480
операции с недвижимостью,10751,780,0.072551
получение образования,3988,369,0.092528
проведение свадьбы,2313,183,0.079118


**Вывод:** Люди, берущие кредит на операции с недвижимостью, с большей вероятностью выплачивают его в срок.

#### Возможные причины появления пропусков в исходных данных.

*Ответ:* Люди намерено не указывали данные, чтобы повысить шанс получения кредита - человеческий фактор
Возможно, столбцы days_employed и total_income зависимы друг от друга, поэтому если в одном столбце отсутствует значение, то в другом его также не будет - технический фактор.

#### Почему заполнение пропусков медианным значением — лучшее решение для количественных переменных.

*Ответ:* Значения количественных переменных могут отличаться друг от друга в несколько раз (одно большое значение может сместить среднее значение данных), поэтому среднее арифметическое не даст действительного среднего значения

### Общий вывод.

**Вывод:** 
    В таблице проекта были выявлены пропуски в столбцах days_employed и total_income и заполнены медианными значениями соответствующих столбцов, выявлены и удалены аномальные значения в столбце children, обработаны и удалены неявные дубликаты в столбце education. Также в таблице были созданы 2 столбца: total_income_category - категории доходов людей, purpose_category - категории целей для кредита.
    По полученным данным можно сказать, что определяющим фактором возврата кредита являются семейное положение и цель кредита.
    Наиболее надежным заемщиком можно назвать человека состоящего в браке, взявшего кредит на операции с недвижимостью, с категорией дохода В. Наиболее же рискованный заемщик - человек, не состоявший в официальном браке, берущий кредит на операции с автомобилем или получение образования, с категорией дохода С.
    рекомендация: При выдаче кредита банку стоит уделять большее внимание семейному положению клиента
    